Order of action
1. Revalue BDI swaps
2. Deisel cost pass through 
3. Bonus price contract freight phosphate
4. Deisel Force Majeure 
5. Spread 
6. Phosphate cargoes

In [ ]:
# 1. Revalue Bdi Swaps 
"""

"""